In [1]:
from IPython.display import Image
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()

import warnings
warnings.simplefilter("ignore")

## EmazeMeGroups

In [ ]:
groups = pd.read_csv('data/EmazeMeGroups.csv')

In [27]:
groups.head()

,EmazeMeGroupID,GroupName,Admin,Privecy,Visibility,priority,varified,AssociatedGroupsJson,ModeratorsJson,DateCreated,DateUpdated,AboutGroup,EmazeSite,GroupCoverUrl,GroupPhotoUrl,SiteThumnail
0,1,BeitarJerusalem,d1f13501-60bf-4dd8-b6e8-c11d2f8eab8c,1,0,0,0,NaN,NaN,27:38.1,27:38.1,Group of Beitar players,https://emaze.me/diogoverdasca,//userscontent2.emaze.com/userinfo//cover/2dc9...,//userscontent2.emaze.com/userinfo//image/7c4d...,//storage.googleapis.com/userscontent2.emaze.c...
1,2,BeitarFans,d1f13501-60bf-4dd8-b6e8-c11d2f8eab8c,1,0,0,0,NaN,NaN,27:38.1,27:38.1,Group for fans of Beitar Jerusalem f.c.,NaN,//userscontent2.emaze.com/userinfo//cover/b397...,//userscontent2.emaze.com/userinfo//image/1e5a...,NaN
2,3,newgroupwithdate2,NaN,0,0,0,0,NaN,NaN,00:36.2,00:36.2,NaN,NaN,NaN,NaN,NaN
3,4,another-group,NaN,0,0,0,0,NaN,NaN,57:01.8,57:01.8,NaN,NaN,NaN,NaN,NaN
4,5,"""""",NaN,0,0,0,0,NaN,NaN,44:39.2,44:39.2,NaN,NaN,NaN,NaN,NaN


In [28]:
groups = groups.rename(columns = {'GroupID':'EmazeMeGroupID'})

## EmazeMeGroupFollowers

In [30]:
group_f = pd.read_csv('data/EmazeMeGroupFollowers.csv')

In [31]:
group_f.head()

,FollowId,FollowerId,EmazeMeGroupId,FollowingSince
0,3,240238a0-ce99-4ea7-ac77-eff09fb5eff2,1,0001-01-01 00:00:00.0000000
1,4,240238a0-ce99-4ea7-ac77-eff09fb5eff2,2,0001-01-01 00:00:00.0000000
2,5,e9477dec-c960-4f11-a563-9781cf8b1c15,1,0001-01-01 00:00:00.0000000
3,6,e9477dec-c960-4f11-a563-9781cf8b1c15,2,0001-01-01 00:00:00.0000000
4,7,93aff917-ec28-4201-b1d2-788e275671b2,1,0001-01-01 00:00:00.0000000


In [32]:
#create a values column for the matrix, make everything 1 since already followed
group_f['following'] = 1

In [33]:
#dictionary for groupid to groupname
item_dict = dict(zip(groups['EmazeMeGroupID'], groups['GroupName']))

## Create sparse FollowerId and Quantity matrix

In [34]:
group_matrix_df = group_f.pivot_table(
    values='following', 
    index='FollowerId', 
    columns='EmazeMeGroupId'
)

In [35]:
group_matrix_df.head()

EmazeMeGroupId,1,2,13,14,15,26,30
FollowerId,,,,,,,
09d32136-ed59-49fd-b553-5c1448aa92fd,NaN,NaN,NaN,NaN,NaN,1.0,NaN
1c1994c7-9bb0-477d-83dd-dc0d6f822c10,NaN,NaN,NaN,NaN,NaN,1.0,NaN
240238a0-ce99-4ea7-ac77-eff09fb5eff2,1.0,1.0,NaN,1.0,1.0,NaN,NaN
2415e720-8d4c-44c7-9cbf-cb1e76dcf4e8,NaN,NaN,NaN,NaN,NaN,1.0,NaN
24c1d879-cb12-4582-87b1-982c3b9ef4e8,NaN,NaN,NaN,NaN,NaN,1.0,NaN


In [10]:
follower_id_list = np.array(group_matrix_df.index.tolist())
group_id_list = np.array(group_matrix_df.columns.tolist())

In [11]:
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import svds

In [12]:
group_mtx = group_matrix_df.fillna(0).values.copy()
group_sparse_mtx = coo_matrix(group_mtx)

## SVD of matrix

In [14]:
U, s, V = svds(group_sparse_mtx, k=1)

## Multiply the decompostion back to get approx matrix

In [15]:
s_diag = np.diag(s)

In [16]:
group_svd = U @ s_diag @ V

In [18]:
group_svd

array([[0.29102584, 0.297855  , 0.16059587, 0.12919113, 0.17990852,
        0.538028  , 0.01708134],
       [0.29102584, 0.297855  , 0.16059587, 0.12919113, 0.17990852,
        0.538028  , 0.01708134],
       [0.48572849, 0.4971265 , 0.26803802, 0.21562282, 0.30027126,
        0.89798049, 0.02850913],
       [0.29102584, 0.297855  , 0.16059587, 0.12919113, 0.17990852,
        0.538028  , 0.01708134],
       [0.29102584, 0.297855  , 0.16059587, 0.12919113, 0.17990852,
        0.538028  , 0.01708134],
       [0.31853276, 0.32600739, 0.17577492, 0.1414019 , 0.19691295,
        0.58888084, 0.01869582],
       [0.29102584, 0.297855  , 0.16059587, 0.12919113, 0.17990852,
        0.538028  , 0.01708134],
       [0.29102584, 0.297855  , 0.16059587, 0.12919113, 0.17990852,
        0.538028  , 0.01708134],
       [0.31853276, 0.32600739, 0.17577492, 0.1414019 , 0.19691295,
        0.58888084, 0.01869582],
       [0.08686825, 0.08890668, 0.04793623, 0.03856223, 0.05370086,
        0.16059587, 0.0

# Rec Function

In [25]:
def recommend(follower_id, num_results=5):
    follower_indx = np.where(follower_id_list == follower_id)[0][0]
    indx_sort = group_svd[follower_indx, :].argsort()[::-1]
    never_buy = group_mtx[follower_indx, :][indx_sort] == 0
    rec_indx = indx_sort[never_buy]
    rec_ids = group_id_list[rec_indx]
    rec_name = [item_dict[x] for x in rec_ids]
    return rec_ids[:num_results], rec_name[:num_results]

In [26]:
rec_ids, rec_name = recommend('09d32136-ed59-49fd-b553-5c1448aa92fd')
print(rec_ids)
print(rec_name)

[ 2  1 15 13 14]
['BeitarFans', 'BeitarJerusalem', 'Israeli Singers', 'Emaze', 'Design and Marketing Team']
